#  Unit 2.4a Using Programs with Data, SQLAlchemy
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- image: /images/python.png
- categories: []
- type: ap
- week: 26

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.  ***Use Debugging through these examples to examine Objects created in Code***.

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?  Provide a defintion of purpose.
    1. Flask app object
    2. SQLAlchemy db object


In [1]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class, purpose and defintion.
    - class User
    - db.Model inheritance
    - _init_ method
    - ```@property```, ```@<column>.setter```
    - create, read, update, delete methods

In [2]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class User(db.Model):
    __tablename__ = 'users'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _dob = db.Column(db.Date)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, uid, password="123qwerty", dob=datetime.today()):
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self.set_password(password)
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts uid from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows uid to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def password(self):
        return self._password[0:10] + "..." # because of security only show 1st characters

    # update password, this is conventional method used for setter
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # check password parameter against stored/encrypted password
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    # dob property is returned as string, a string represents date outside object
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # dob setter, verifies date type before it is set or default to today
    @dob.setter
    def dob(self, dob):
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob
    
    # age is calculated field, age is returned according to date of birth
    @property
    def age(self):
        today = datetime.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "dob": self.dob,
            "age": self.age,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", uid="", password=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object
    2. User Object Constructors
    3. Try / Except 


In [27]:
"""Database Creation and Testing """


# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob=datetime(1847, 2, 11))
        u2 = User(name='Nikola Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')
        u5 = User(name='Indiana Jones', uid='indi', dob=datetime(1920, 10, 21))
        u6 = User(name='Marion Ravenwood', uid='raven', dob=datetime(1921, 10, 21))


        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers()

Created new uid toby
Created new uid niko
Created new uid lex
Created new uid whit
Created new uid indi
Created new uid raven


## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
    1. User.query.filter_by
    2. user.password

In [4]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_uid(uid):
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    return user # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False
        
#check_credentials("indi", "123qwerty")

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
    1. user.find_by_uid() and try/except
    2. user = User(...)
    3. user.dob and try/except
    4. user.create() and try/except

In [5]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    password = input("Enter your password")
    
    # Initialize User object before date
    user = User(name=name, 
                uid=uid, 
                password=password
                )
    
    # create user.dob, fail with today as dob
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

Found
 {'id': 7, 'name': 'Ethan', 'uid': 'ethan123', 'dob': '05-09-2006', 'age': 16}


## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. User.query.all
    2. json_ready assignment, google List Comprehension

In [7]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'Thomas Edison',
  'uid': 'toby',
  'dob': '02-11-1847',
  'age': 176},
 {'id': 2,
  'name': 'Nikola Tesla',
  'uid': 'niko',
  'dob': '03-14-2023',
  'age': 0},
 {'id': 3,
  'name': 'Alexander Graham Bell',
  'uid': 'lex',
  'dob': '03-14-2023',
  'age': 0},
 {'id': 4,
  'name': 'Eli Whitney',
  'uid': 'whit',
  'dob': '03-14-2023',
  'age': 0},
 {'id': 5,
  'name': 'Indiana Jones',
  'uid': 'indi',
  'dob': '10-21-1920',
  'age': 102},
 {'id': 6,
  'name': 'Marion Ravenwood',
  'uid': 'raven',
  'dob': '10-21-1921',
  'age': 101},
 {'id': 7, 'name': '', 'uid': 'ethan123', 'dob': '03-14-2023', 'age': 0}]

# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.

# My Hacks

## Lesson Notes

- db.Model: Inheritance - capable to make the class utilize database attributes and methods (getters and setters) 
- Init method instantiate an object from user class
- Setters allow attributes to be changed
- Getters enable the retrieval of a given object

In [2]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///cars.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)


In [62]:
from sqlalchemy import Column, Integer, Text, String, Boolean
from sqlalchemy.exc import IntegrityError

class Car(db.Model):
    __tablename__ = "cars"
    __table_args__ = {'extend_existing': True}
    id = db.Column(db.Integer, primary_key=True)
    _make = db.Column(db.String(255), nullable=False, unique = False)
    _model = db.Column(db.String(255), nullable=False, unique = False)
    _price = db.Column(db.Integer, nullable=False, unique = False)
    _year = db.Column(db.Integer, nullable=False, unique = False)
    _desc = db.Column(db.String(255), nullable=False, unique = True)
    _engine = db.Column(db.String(255), nullable=False, unique = False)
    _body_style = db.Column(db.String(255), nullable=False, unique = False)
    _owner = db.Column(db.String(255), nullable=False, unique = False)
    
    def __init__(self, make, model, price, year, desc, body_style, engine, owner):
        # Adding instance attributes
        self._make = make
        self._model = model
        self._price = price
        self._year = year
        self._desc = desc
        self._body_style = body_style
        self._engine = engine
        self._owner = owner
        
    # Add getters and setters for make, model, price, year
    @property
    def make(self):
        return self._make
    
    @make.setter
    def make(self, make):
        self._make = make

    @property
    def model(self):
        return self._model
    
    @model.setter
    def model(self, model):
        self._model = model

    @property
    def price(self):
        return self._price
    
    @price.setter
    def price(self, price):
        self._price = price

    @property
    def year(self):
        return self._year
    
    @year.setter
    def year(self, year):
        self._year = year
    
    @property
    def desc(self):
        return self._desc
    
    @desc.setter
    def desc(self, desc):
        self._desc = desc

    @property
    def body_style(self):
        return self._body_style
    
    @body_style.setter
    def body_style(self, body_style):
        self._body_style = body_style

    @property
    def engine(self):
        return self._engine
    
    @engine.setter
    def engine(self, engine):
        self._engine = engine

    @property
    def owner(self):
        return self._owner

    @owner.setter
    def owner(self, owner):
        self._owner = owner
    

    def dictionary(self):
        dict = {
            "make" : self.make,
            "model" : self.model,
            "price" : self.price,
            "year" : self.year,
            "desc" : self.desc,
            "body_style" : self.body_style,
            "engine" : self.engine,
            "owner" : self.owner
        }
        return dict 

    def __str__(self):
        return json.dumps(self.dictionary)

    def create(self):
        try:
            # creates a Car object from Car(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id" : self.id,
            "make" : self.make,
            "model" : self.model,
            "price" : self.price,
            "year" : self.year,
            "desc" : self.desc,
            "body_style" : self.body_style,
            "engine" : self.engine,
            "owner" : self.owner        
        }

    # CRUD update: updates attributes
    # returns self
    def update(self, make="", model="", price="", year="", desc="", body_style="", engine="", owner=""):
        """only updates values with length"""
        if len(make) > 0:
            self.make = make
        if len(model) > 0:
            self.model = model
        if price > 0:
            self.price(price)
        if year > 0:
            self.year(year)
        if desc >= 0:
            self.desc(desc)
        if len(body_style) > 0:
            self.body_style(body_style)
        if len(engine) > 0:
            self.engine(engine)
        if len(owner) > 0:
            self.owner(owner)
        db.session.commit()
        return self  
    
    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None

# Function to initialize the Cars
def initCars():
    with app.app_context():
        """Create database and tables"""
        # db.init_app(app)
        db.create_all()

        """Data for table"""
        car1 = Car(make="BMW", model="2 Series", price=34000, year=2021, desc="The BMW 2 series is a stylish sports coupe that drives just as well as it looks.", body_style="coupe", engine="2.0L four-cylinder", owner="John Doe")

        cars = [car1]

        """Builds sample user/note(s) data"""
        for car in cars:
            try:
                car.create()
            except IntegrityError:
                '''fails with bad or duplicate data'''
                db.session.remove()
                print(f"Records exist, duplicate car, or error: {car.id}")

initCars()

## Create a new car

In [4]:
import sqlite3

database = 'instance/cars.db'

def create():
    make = input("Enter the make of the car:")
    model = input("Enter the model:")
    price = input("Enter the price:")
    year = input("Enter the year:")
    desc = input("Enter a description of the car:")
    body_style = input("Enter the body style of the car:")
    engine = input("Enter the engine type:")
    owner = input("Who's the owner of the car?")
        
    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    try:
        # Execute an SQL command to insert data into a table
        cursor.execute("INSERT INTO cars (_make, _model, _price, _year, _desc, _body_style, _engine, _owner) VALUES (?, ?, ?, ?, ?, ?, ?, ?)", (make, model, price, year, desc, body_style, engine, owner))
        
        # Commit the changes to the database
        conn.commit()
        print(f"A new car has been added")
                
    except sqlite3.Error as error:
        print("Error while executing the INSERT:", error)


    # Close the cursor and connection objects
    cursor.close()
    conn.close()
    
create()

A new car has been added


## Read existing records

In [50]:
# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = Car.query.all()
    json_ready = [car.read() for car in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 1,
  'make': 'BMW',
  'model': '2 Series',
  'price': 34000,
  'year': 2021,
  'desc': 'The BMW 2 series is a stylish sports coupe that drives just as well as it looks.',
  'body_style': 'coupe',
  'engine': '2.0L four-cylinder',
  'owner': 'John Doe'},
 {'id': 2,
  'make': 'Lotus',
  'model': 'Evora',
  'price': 68000,
  'year': 2019,
  'desc': 'NEW DESCRIPTION!!',
  'body_style': 'Coupe',
  'engine': 'V6',
  'owner': 'John Doe II'},
 {'id': 3,
  'make': 'Toyota',
  'model': 'Sienna',
  'price': 40000,
  'year': 2019,
  'desc': 'Reliable Van',
  'body_style': 'Van',
  'engine': 'Van',
  'owner': 'Johnny Doe'},
 {'id': 4,
  'make': '',
  'model': '',
  'price': '',
  'year': '',
  'desc': '',
  'body_style': '',
  'engine': '',
  'owner': ''}]

## Update an existing record

In [33]:
import sqlite3

database = 'instance/cars.db'

# Take the inputs for each attribute
def update():
    id = input("Enter the id of the car:")
    make = input("Enter the make of the car:")
    model = input("Enter the model:")
    price = input("Enter the price:")
    year = input("Enter the year:")
    desc = input("Enter a description of the car:")
    body_style = input("Enter the body style of the car:")
    engine = input("Enter the engine type:")
    owner = input("Who's the owner of the car?")

    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    try:
        # Execute an SQL command to update data in a table
        # Trying to add dob right now...
        cursor.execute("UPDATE cars SET _make = ?, _model = ?, _price = ?, _year = ?, _desc = ?,  _body_style = ?,  _engine = ?,  _owner = ?  WHERE id = ?", (make, model, price, year, desc, body_style, engine, owner, id))
        if cursor.rowcount == 0:
            # Car ID not found
            print(f"No id was found in the table")
        
        # Commit new data
        else: 
            print(f"Car with the id {id} has been updated")
            conn.commit()

    except sqlite3.Error as error:
        print("Error while updating the record:", error)
    
    # Close the cursor and connection objects
    cursor.close()
    conn.close()
    
update()


Car with the id 2 has been updated


### Check the update

> Read to check if the record has truly been updated 

In [51]:
read()

[{'id': 1,
  'make': 'BMW',
  'model': '2 Series',
  'price': 34000,
  'year': 2021,
  'desc': 'The BMW 2 series is a stylish sports coupe that drives just as well as it looks.',
  'body_style': 'coupe',
  'engine': '2.0L four-cylinder',
  'owner': 'John Doe'},
 {'id': 2,
  'make': 'Lotus',
  'model': 'Evora',
  'price': 68000,
  'year': 2019,
  'desc': 'NEW DESCRIPTION!!',
  'body_style': 'Coupe',
  'engine': 'V6',
  'owner': 'John Doe II'},
 {'id': 3,
  'make': 'Toyota',
  'model': 'Sienna',
  'price': 40000,
  'year': 2019,
  'desc': 'Reliable Van',
  'body_style': 'Van',
  'engine': 'Van',
  'owner': 'Johnny Doe'},
 {'id': 4,
  'make': '',
  'model': '',
  'price': '',
  'year': '',
  'desc': '',
  'body_style': '',
  'engine': '',
  'owner': ''}]

## Delete an existing record

In [67]:
import sqlite3

def delete():

    id = input("Select the ID of the car you wish to remove: ")
    
    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    # Validate the input
    try:
        id = int(id)
    except ValueError:
        print("Invalid input: ID must be an integer.")
        return

    try:
        # Use a parameterized query to prevent SQL injection
        c = conn.cursor()
        c.execute("DELETE FROM cars WHERE id = ?", (id,))
        conn.commit()
        print(f"Row with ID {id} has been deleted.")
    except sqlite3.Error as e:
        print(f"Error deleting row with ID {id}: {e}")
    finally:
        # Close the connection
        cursor.close
        conn.close()

delete()

Row with ID 2 has been deleted.


### Run a read to check if the record has truly been deleted

In [68]:
read()

[{'id': 1,
  'make': 'BMW',
  'model': '2 Series',
  'price': 34000,
  'year': 2021,
  'desc': 'The BMW 2 series is a stylish sports coupe that drives just as well as it looks.',
  'body_style': 'coupe',
  'engine': '2.0L four-cylinder',
  'owner': 'John Doe'},
 {'id': 3,
  'make': 'Toyota',
  'model': 'Sienna',
  'price': 40000,
  'year': 2019,
  'desc': 'Reliable Van',
  'body_style': 'Van',
  'engine': 'Van',
  'owner': 'Johnny Doe'},
 {'id': 4,
  'make': '',
  'model': '',
  'price': '',
  'year': '',
  'desc': '',
  'body_style': '',
  'engine': '',
  'owner': ''}]